<a href="https://colab.research.google.com/github/Rattapon-Insa/time-series/blob/main/LSTM_on_NIFTY_50_Stock_Market_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rohanrao/nifty50-stock-market-data

In [ ]:
! unzip nifty50-stock-market-data.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('HCLTECH.csv')

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'])
df.dropna()
df.set_index(df['Date'], inplace = True)
df = df[['Prev Close']]['2013-01-01':'2013-12-02']

In [ ]:
df1 = df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range= (0,1))
df1_scaled_train = sc.fit_transform(df1)

In [ ]:
df1_scaled_train.shape[0]

In [ ]:
# test train split

train = df1_scaled_train[:180]
validate = df1_scaled_train[180:]


In [ ]:
"""setup window for training. Since our dataset size is 230, let's set 
the window to 10% of size as the starting point"""
hops = int(0.1*train.shape[0])
total_len = train.shape[0]
X= []
Y= []

for i in range(hops, total_len):
  X.append(train[i-hops:i])
  Y.append(train[i])

X = np.array(X)
Y = np.array(Y)

In [ ]:
# Build model

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten, Dropout


model = Sequential()
model.add(LSTM(units = 50, return_sequences= True
               , input_shape = (X.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units =50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')


In [ ]:
model.fit(X,Y, epochs = 100, batch_size = 64)

In [ ]:
X_test = []

for i in range(hops, validate.shape[0]):
  X_test.append(validate[i-hops:i])

X_test = np.array(X_test)

In [ ]:
X_test.shape

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_pred = sc.inverse_transform(y_pred)
y_pred.shape

In [ ]:
validate = sc.inverse_transform(validate)

In [ ]:
validate = validate[50-32:]

In [ ]:
plt.plot(y_pred, color = 'red', label = 'Predict')
plt.plot(validate, color = 'blue', label = 'Actual')
plt.legend()
plt.show()